In [37]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import json

In [39]:
def get_status(logs):
    for log in logs:
        if log['message']:
            d = json.loads(log['message'])
            try:
                content_type = 'text/html' in d['message']['params']['response']['headers']['content-type']
                response_received = d['message']['method'] == 'Network.responseReceived'
                if content_type and response_received:
                    return d['message']['params']['response']['status']
            except:
                pass

def preprocess(line):
    return ' '.join(line.replace('\n','').replace('\r','').replace('\xa0','').strip().split())

In [49]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-blink-features=AutomationControlled')
chrome_options.set_capability('goog:loggingPrefs', {'performance': 'ALL'})
driver = webdriver.Chrome(options = chrome_options)

In [52]:
links = input().split()
responses = []
selenium = []
status = 0
for link in links:
    try:
        page = requests.get(link)
        status = page.status_code
    except:
        pass
    if status == 200:
        page.encoding = 'utf-8'
        responses.append((link, page.text))
    else:
        page.encoding = 'utf-8'
        selenium.append(link)
    print(link, status, sep=' :')
    status = 0


 https://alfabank.ru/ https://www.tinkoff.ru/ http://www.sberbank.ru/ru/person https://www.vtb.ru/ https://spbbank.ru/ https://business.rsb.ru/ https://www.gazprombank.ru/ https://www.raiffeisen.ru/ https://www.rosneft.ru/ https://lukoil.ru/ https://www.x5.ru/ru/ https://www.gazprom.ru/ https://www.gkm.ru/ https://www.tatneft.ru/ https://moskva.mts.ru/personal https://www.tplusgroup.ru/ https://www.evraz.com/ru/ https://www.eurochem.ru/


https://alfabank.ru/ :200
https://www.tinkoff.ru/ :200
http://www.sberbank.ru/ru/person :200
https://www.vtb.ru/ :403
https://spbbank.ru/ :200
https://business.rsb.ru/ :200
https://www.gazprombank.ru/ :0
https://www.raiffeisen.ru/ :200
https://www.rosneft.ru/ :403
https://lukoil.ru/ :502
https://www.x5.ru/ru/ :200
https://www.gazprom.ru/ :200
https://www.gkm.ru/ :0
https://www.tatneft.ru/ :0
https://moskva.mts.ru/personal :200
https://www.tplusgroup.ru/ :403
https://www.evraz.com/ru/ :200
https://www.eurochem.ru/ :200


In [53]:
len(responses)

11

In [54]:
selenium

['https://www.vtb.ru/',
 'https://www.gazprombank.ru/',
 'https://www.rosneft.ru/',
 'https://lukoil.ru/',
 'https://www.gkm.ru/',
 'https://www.tatneft.ru/',
 'https://www.tplusgroup.ru/']

In [21]:
menu_links = []
for response in responses:
    html = BeautifulSoup(response[1], 'html.parser')
    a_tags = html.find_all('a')
    print(response[0],' ',[preprocess(i.text) for i in a_tags])
    menu_links.append({response[0]: links})

https://alfabank.ru/   ['', 'Частным лицам', 'Малому бизнесу иИП', 'Корпорациям', 'Финансовым организациям', 'А-КЛУБ', 'Карты', 'Вклады', 'Кредиты', 'Ипотека', 'Инвестиции', 'Премиум', 'Ещё', 'Кредитные карты', 'Целый год без% 🔥', 'Alfa Travel', 'Как получить отсрочку покредитам', 'Полезные статьи', 'Дебетовые карты', 'Альфа-Карта 🔥', 'Альфа-Карта Premium', 'Детская карта', 'Alfa Travel', 'Платёжные стикеры', 'Специальные условия', 'Зарплатные карты', 'Зарплата каждый день', 'Индивидуальное зарплатное предложение', 'Вклады', 'Альфа-Вклад Максимальный🔥', 'Альфа-Вклад', 'Накопительные счета', 'Альфа-Счёт', 'Мили «Аэрофлот Бонус» заостаток насчёте', 'Бесплатные сервисы длянакоплений', 'Копилка для зарплаты', 'Автопополнение накопительного счёта', 'Альфа-Вклад МаксимальныйОткрыть', 'Все кредиты', 'Кредит наличными', 'Кредит наавтомобиль', 'Кредит под залог налюбые цели', 'Рефинансирование кредита', 'Кредитные карты', 'Кредит нановый электромобиль⚡', 'Полезное', 'Кредитный калькулятор', 'По

In [59]:
for link in selenium:
    driver.get(link)
    logs = driver.get_log('performance')
    print(link,' : ',get_status(logs))

https://www.vtb.ru/  :  200
https://www.gazprombank.ru/  :  200
https://www.rosneft.ru/  :  200
https://lukoil.ru/  :  None
https://www.gkm.ru/  :  200
https://www.tatneft.ru/  :  200
https://www.tplusgroup.ru/  :  200
